graph TD
    subgraph "A. Developer & CI/CD"
        DEV[🧑‍💻 Developer] -- 1. Push Code --> GIT[📁 Git (GitHub/GitLab)]
        GIT -- 2. Trigger --> CICD[🚀 CI/CD Pipeline]
        CICD -- 3. Build & Test --> CICD
        CICD -- 4. Push Image --> REG[🐳 Container Registry]
        CICD -- 5. Deploy --> K8S[🌐 K8s Cluster (Local/Cloud)]
    end

    subgraph "B. Storage & Data (Bên ngoài K8s)"
        DB[(🗃️ PostgreSQL DB)]
        S3[📦 MinIO / AWS S3]
    end

    subgraph "C. K8s Cluster (Runtime)"
        subgraph "Luồng 1: User Inference (Online)"
            USER[👨‍👩‍👧‍👦 User] --> FE[Frontend (Streamlit)]
            FE --> API[Backend API (FastAPI)]
            API --> MOCK[Mock Use Case (Pod CPU)]
            API --> M1[Model 1 (Pod GPU)]
            API --> M2[Model 2 (Pod GPU)]
            API --> M3[Model 3 (Pod GPU)]
        end

        subgraph "Luồng 2: MLOps Tools (Internal)"
            MLFLOW[📊 MLflow Server]
            AIRFLOW[🌀 Airflow]
            PROM[📈 Prometheus]
            GRA[📊 Grafana]
        end

        subgraph "Luồng 3: Model Training (Offline Job)"
            AIRFLOW -- 6. Schedule Job --> K8S
            K8S -- 7. Tạo Pod --> JOB[Train Job (Pod GPU)]
            JOB -- 8. Lấy data --> S3
            JOB -- 9. Log/Save Model --> MLFLOW
            JOB -- 9. Log/Save Model --> S3
            M1 -- Tải model khi khởi động --> MLFLOW
            M1 -- Tải model khi khởi động --> S3
        end
    end

    %% Kết nối Tooling với Storage
    MLFLOW -- "Lưu Metadata (Experiments)" --> DB
    MLFLOW -- "Lưu Artifacts (Models)" --> S3
    AIRFLOW -- "Lưu Metadata (DAGs)" --> DB

    %% Kết nối Monitoring
    PROM -- Scrape metrics --> API
    PROM -- Scrape metrics --> M1
    PROM -- Scrape metrics --> MOCK
    PROM -- Scrape metrics --> K8S
    GRA -- Query --> PROM

In [ ]:
Chắc chắn rồi. Vì tôi không thể tạo ra một file ảnh, tôi sẽ thể hiện sơ đồ kiến trúc này dưới dạng text trực quan, chia thành các khối (Zone) và các luồng (Flow) rõ ràng.

Đây là kiến trúc MLOps "Cloud-Ready" hoàn chỉnh của bạn:

-----

### 🗺️ Sơ đồ Kiến trúc MLOps "Cloud-Ready"

Kiến trúc này được chia thành 3 khu vực chính:

1.  **Zone 1: CI/CD & Development** (Tự động hóa)
2.  **Zone 2: Storage & Databases** (Lưu trữ bên ngoài)
3.  **Zone 3: K8s Cluster** (Nơi chạy ứng dụng)

-----

### [ ZONE 1: CI/CD & DEVELOPMENT ]

> **Mục tiêu:** Tự động hóa việc build, test và deploy code mới.

**(1) 🧑‍💻 Developer**
│
└─ `(1. Push Code)` ─\> **(2) 📁 Git (GitHub / GitLab)**
│
└─ `(2. Trigger)` ─\> **(3) 🚀 CI/CD Pipeline (Actions, Jenkins)**
│
├─ `(3. Build & Test Code)`
│
├─ `(4. Push Image)` ─\> **(4) 🐳 Container Registry**
│
└─ `(5. Deploy/Update)` ─\> **[ ZONE 3: K8s Cluster ]**

-----

### [ ZONE 2: STORAGE & DATABASES ]

> **Mục tiêu:** Lưu trữ dữ liệu lâu dài (Persistent Storage), độc lập với K8s.

  * **(A) 🗃️ PostgreSQL Database**

      * `<- (Lưu metadata)` ── **[ 🌀 Airflow ]** (từ Zone 3)
      * `<- (Lưu metadata)` ── **[ 📊 MLflow ]** (từ Zone 3)

  * **(B) 📦 MinIO / S3 (Object Storage)**

      * `<- (Lưu artifacts/models)` ── **[ 📊 MLflow ]** (từ Zone 3)
      * `<- (Lưu data/models)` ── **[ 🤖 Train Job Pod ]** (từ Zone 3)
      * `-> (Đọc data/models)` ── **[ 🤖 Train Job Pod ]** (từ Zone 3)

-----

### [ ZONE 3: K8s CLUSTER (Local / AWS EKS) ]

> **Mục tiêu:** Nơi chạy tất cả ứng dụng, training và MLOps tools.

```
+-----------------------------------------------------------------------------------------+
|                                 🌐 K8s CLUSTER                                          |
|                                                                                         |
|   +--------------------------+                                                          |
|   |   LUỒNG 1: INFERENCE     |                                                          |
|   |  (Phục vụ người dùng)    |                                                          |
|   |                          |     +------------------+     +-----------------------+   |
|   |  [ 👨‍👩‍👧‍👦 User ]           |     | [ 🚀 Backend API ] | --> | [ 📝 Mock Pod (CPU) ] |   |
|   |      |                   |     |   (FastAPI)      |     +-----------------------+   |
|   |      v                   |     |                  |                               |
|   |  [ 🖥️ Frontend ]<--------+     |                  |     +-----------------------+   |
|   |  (Streamlit)             |     |                  | --> | [ 🧠 Model 1 (GPU)  ] |   |
|   |                          |     +------------------+     +-----------------------+   |
|   |                                        |                 +-----------------------+   |
|   |                                        |             --> | [ 🧠 Model 2 (GPU)  ] |   |
|   |                                        |                 +-----------------------+   |
|   +--------------------------+             |                                           |
|                                            |                                           |
|   +--------------------------+             |                                           |
|   |    LUỒNG 2: TRAINING     |<------------+                                           |
|   |     (Tự động bởi Airflow) |                                                          |
|   |                          |     +------------------+     +-----------------------+   |
|   | [ 🌀 Airflow ]           |     |                  |     | [ 🤖 Train Job (GPU)] |   |
|   |      |                   |     |                  |     |      (Chạy và Tắt)    |   |
|   |      └─(6. Schedule Job)─+---->(K8s API)          |     |          |            |   |
|   |                          |     (7. Tạo Pod)       |     |          v            |   |
|   |                          |                        |     |  (8. Log/Save Model)  |   |
|   |                          |                        |     |          |            |   |
|   +--------------------------+                        |     |          v            |   |
|                                                       |     |  [ 📊 MLflow ]        |   |
|   +---------------------------------------------------+     |          |            |   |
|   |    LUỒNG 3: MLOps TOOLS & MONITORING                |     |          v            |   |
|   |                                                   |     +----[ 📦 MinIO / S3 ]<--+   |
|   | [ 📊 MLflow Server ] -----------------------------+                               |
|   | [ 🌀 Airflow Server ]                               |                               |
|   |                                                   |                               |
|   | [ 📈 Prometheus ] -- (Scrape Metrics) --> (Tất cả các Pod)                        |
|   | [ 📊 Grafana ]    -- (Query) ----------> [ 📈 Prometheus ]                        |
|   |                                                   |                               |
+-----------------------------------------------------------------------------------------+
```